In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image

from os import listdir

from torchvision import transforms, utils

from torchvision import transforms 

import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm

import glob

In [4]:
from tqdm import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()

In [5]:
def stripping(liste):
    return [i.strip() for i in liste]

In [8]:
df=pd.read_csv('data/over.csv',converters={'list_alle': lambda x: x[1:-1].strip('][').replace("'","").strip().split(','),
'list_normen': lambda x: x[1:-1].strip('][').replace("'","").strip().split(','),
'list_gesamt': lambda x: x[1:-1].strip('][').replace("'","").strip().split(',')
})

df['list_alle']=df['list_alle'].progress_apply(stripping)
df['list_normen']=df['list_normen'].progress_apply(stripping)
df['list_gesamt']=df['list_gesamt'].progress_apply(stripping)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df=df.reset_index(drop=True)

  0%|          | 0/19500 [00:00<?, ?it/s]

  0%|          | 0/19500 [00:00<?, ?it/s]

  0%|          | 0/19500 [00:00<?, ?it/s]

In [9]:
df.head()

,text,topic,gender,age,topic_num,percent,list_alle,list_normen,list_gesamt
0,This blog is being posted due to the fact that...,Student,male,14,7,0.4414019715224534,"[0.0, 0.0, 0.16700000000000004, 0.0, 0.021, 0....","[0.0, 0.0, 0.0, 0.0, 0.055999999999999994, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001, 0.0, 0.0..."
1,So I have a big fucking interview tomorrow for...,Student,male,15,20,0.3124452234881683,"[0.176, 0.0, 0.083, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[0.0, 0.849, 0.0, 0.0, 0.023, 0.0, 0.153000000...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001, 0.0, 0.0..."
2,I was reminded just now of the time Ashley and...,Student,female,17,27,0.20397382322794297,"[1.863999999999999, 0.05, 0.5950000000000003, ...","[0.0, 5.121999999999999, 0.0, 0.0, 0.417000000...","[0.004, 0.0, 0.0, 0.018000000000000002, 0.0, 0..."
3,I was checking up on my cousin Dylan and Fanni...,Student,female,23,20,0.2786579683131407,"[0.0, 0.0, 0.08300000000000002, 0.08, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.069, 0.0, 0.042, 0.0, 0...","[0.0, 0.0, 0.003, 0.006, 0.0, 0.0, 0.0, 0.0, 0..."
4,for the NME interview click urlLink part 1 and...,Student,female,23,31,0.9468664850136239,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.014, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001, 0.0, 0.0..."


In [37]:
over=pd.DataFrame(data=df['list_alle'].tolist(),columns=[f'Topic_{x}' for x in range(len(df['list_alle'][0]))])

In [38]:
over.head()

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,Topic_9,...,Topic_28,Topic_29,Topic_30,Topic_31,Topic_32,Topic_33,Topic_34,Topic_35,Topic_36,Topic_37
0,0.0,0.0,0.16700000000000004,0.0,0.021,0.0,0.0,0.403,0.0,0.022,...,0.0,0.15600000000000003,0.0,0.0,0.022,0.0,0.0,0.0,0.0,0.0
1,0.176,0.0,0.083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.016,0.087,0.008,0.112,0.0,0.0
2,1.863999999999999,0.05,0.5950000000000003,0.0,0.0,0.189,0.216,0.0,0.0,0.0,...,0.102,0.0,0.07,0.0,0.0,0.033,0.0,0.14800000000000002,0.064,0.136
3,0.0,0.0,0.08300000000000002,0.08,0.0,0.056,0.0,0.41700000000000004,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.112,0.017,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.39,0.0,0.0,0.0,0.0,0.0,0.04


In [39]:
over = over.astype(float)

In [21]:
from sklearn.preprocessing import StandardScaler, RobustScaler
for cols in over.columns:
    over[cols] = StandardScaler()

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,Topic_9,...,Topic_28,Topic_29,Topic_30,Topic_31,Topic_32,Topic_33,Topic_34,Topic_35,Topic_36,Topic_37
0,1.000000,1.000000,0.845083,1.00000,0.980038,1.000000,1.00000,0.875000,1.0,0.992926,...,1.00000,0.826281,1.000000,1.000000,0.969144,1.00000,1.000000,1.000000,1.000000,1.000000
1,0.961614,1.000000,0.923006,1.00000,1.000000,1.000000,1.00000,1.000000,1.0,1.000000,...,1.00000,1.000000,1.000000,1.000000,0.977560,0.96396,0.999614,0.856410,1.000000,1.000000
2,0.593457,0.966125,0.448052,1.00000,1.000000,0.945407,0.92925,1.000000,1.0,1.000000,...,0.92522,1.000000,0.987879,1.000000,1.000000,0.98633,1.000000,0.810256,0.977778,0.875000
3,1.000000,1.000000,0.923006,0.91658,1.000000,0.983824,1.00000,0.870658,1.0,1.000000,...,1.00000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,0.856410,0.994097,1.000000
4,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.00000,1.000000,1.0,1.000000,...,1.00000,1.000000,1.000000,0.913043,1.000000,1.00000,1.000000,1.000000,1.000000,0.963235


In [41]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
ct = ColumnTransformer([
        ('somename', StandardScaler(), [x for x in over.columns])
    ], remainder='passthrough')

columns_df=ct.fit_transform(over)

In [42]:
columns_df[0:10]

array([[-0.69923967, -0.42413485,  0.01777494, -0.23476964,  0.07850612,
        -0.35338029, -0.29028851,  1.87495284, -0.28890936,  0.06836893,
        -0.25390775, -0.35656502, -0.43895174, -0.350449  , -0.28958754,
        -0.28931603, -0.45513628, -0.22033514, -0.39279358, -0.33107604,
        -0.72484534, -0.2757282 , -0.38179668, -0.21600785, -0.69222581,
        -0.39818937, -0.29866706, -0.67227126, -0.38197017,  4.4987989 ,
        -0.27601024, -0.42915718,  0.2026593 , -0.40476083, -0.16318384,
        -0.3358712 , -0.27033806, -0.31134044],
       [-0.19635853, -0.42413485, -0.49935331, -0.23476964, -0.3939983 ,
        -0.35338029, -0.29028851, -0.30878177, -0.28890936, -0.2787965 ,
        -0.25390775, -0.35656502, -0.43895174, -0.56373392, -0.28958754,
        -0.28931603, -0.30073974, -0.22033514, -0.39279358, -0.33107604,
         0.42287309,  1.96268901, -0.38179668, -0.21600785, -0.31553623,
         0.80053995, -0.29866706, -0.6686314 , -0.38197017, -0.31870598,
   

In [43]:
torch.from_numpy(columns_df).float()

tensor([[-0.6992, -0.4241,  0.0178,  ..., -0.3359, -0.2703, -0.3113],
        [-0.1964, -0.4241, -0.4994,  ...,  2.1587, -0.2703, -0.3113],
        [ 4.6267,  0.2539,  2.6527,  ...,  2.9605,  0.6281,  2.5791],
        ...,
        [-0.6392, -0.4241, -0.3885,  ..., -0.3359, -0.2703, -0.3113],
        [-0.6992, -0.4241, -1.0103,  ..., -0.3359, -0.0317, -0.3113],
        [-0.0649, -0.4241, -0.0376,  ..., -0.3359, -0.2703,  1.3252]])

--> neuronales netzt --> den tensor input --> mit convid 2--> basiernd davon ergebnisse bekommen 

In [58]:
df['topic_num'] = pd.factorize(df['topic'])[0]

In [65]:
scaler = StandardScaler()
scaler.fit_transform(df['topic_num'].tolist().reshape(1, -1))


AttributeError: 'list' object has no attribute 'reshape'

In [66]:
col_names = ['topic_num']
features = df[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
scaled_features = pd.DataFrame(features, columns = col_names)

In [70]:
scaled_features.head()

,topic_num
0,-1.688194
1,-1.688194
2,-1.688194
3,-1.688194
4,-1.688194


In [74]:
torch.FloatTensor(scaled_features['topic_num'].tolist())[0]


tensor(-1.6882)

In [46]:
class ConvolutionalNeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
    self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
    self.fc1 = nn.Linear(128*28*28, 128)
    self.fc2 = nn.Linear(128, 10)

  def forward(self, x):
    x = torch.relu(self.conv1(x))
    x = torch.relu(self.conv2(x))
    x = torch.relu(self.conv3(x))
    x = x.view(-1, 128*28*28)
    x = torch.relu(self.fc1(x))
    x = self.fc2(x)
    return x

# create a tensor input list


In [47]:
model = ConvolutionalNeuralNetwork()
print(model)

ConvolutionalNeuralNetwork(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=100352, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [ ]:
tensor_list=torch.from_numpy(columns_df).float()

In [54]:
torch.from_numpy(columns_df).float()

19500

In [76]:
learning_rate = 0.01
momentum = 0.9
n_epochs = 5

optimizer = optim.SGD(model.parameters(), lr=learning_rate,momentum=momentum)

loss_func = torch.nn.MSELoss()

In [77]:
def train(columns_df,scaled_features,epoch):
    batch_idx = 0
    for row in range(len(torch.from_numpy(columns_df).float())):
        features = torch.from_numpy(columns_df).float()[row]
        price = torch.FloatTensor(scaled_features['topic_num'].tolist())[row]

        if torch.cuda.is_available():
            image = image.cuda()
            features = features.cuda()
            
        output = model(features)
        output = output.reshape(5)
        loss = loss_func(output, price)
        optimizer.zero_grad()  
        loss.backward()
        optimizer.step()
    

In [78]:
for epoch in range(n_epochs):
    # train 
    train(columns_df,scaled_features,epoch)

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [38]